## Data

Спочатку завантажуємо вектори ConceptNet Numberbutch.

In [8]:
!curl https://conceptnet.s3.amazonaws.com/downloads/2017/numberbatch/numberbatch-en-17.06.txt.gz --output numberbatch-en-17.06.txt.gz

In [35]:
!gunzip numberbatch-en-17.06.txt.gz

In [1]:
from gensim.models import KeyedVectors

numberbatch = KeyedVectors.load_word2vec_format("numberbatch-en-17.06.txt", binary=False)

In [3]:
numberbatch.most_similar(['1st'])

[('2nd', 0.8383364677429199),
 ('3rd', 0.7933375835418701),
 ('4th', 0.763849139213562),
 ('first', 0.7570372819900513),
 ('5th', 0.7545791864395142),
 ('6th', 0.7477511167526245),
 ('7th', 0.7396888732910156),
 ('8th', 0.7219843864440918),
 ('9th', 0.6965588331222534),
 ('oneth', 0.6686497926712036)]

Потім парсимо тренувальні і тестові датасети.

In [4]:
from collections import namedtuple

Phrase = namedtuple('Phrase', 'original candidate label')
Token = namedtuple('Token', 'text tags')

def split_tokens(sent):
    tokens = []
    for token in sent.split():
        tags = token.split('/')
        tokens.append(Token(tags[0].lower(), tuple(tags[1:])))
    return tokens


def readData(filename, eval_label, ignoreNone):
    data = []
    with open(filename) as f:
        for line in f:
            fields = line.strip().split('\t')
            if len(fields) == 7:
                (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = fields
            else:
                continue
            label = eval_label(judge)
            if ((label is None) and ignoreNone):
                continue
            data.append(Phrase(split_tokens(origsenttag), split_tokens(candsenttag), label))
    
    return data
                
def eval_amt_label(label):
    nYes = eval(label)[0]            
    
    if nYes >= 3:
        return True
    elif nYes <= 1:
        return False
    
    return None

def eval_expert_label(label):
    nYes = int(label[0])
    
    if nYes >= 4:
        return True
    elif nYes <= 2:
        return False
    
    return None


def readTrainData(filename):
    return readData(filename, eval_amt_label, True)

def readTestData(filename):
    return readData(filename, eval_expert_label, False)

In [5]:
train_data = readTrainData("SemEval-PIT2015-py3/data/train.data")
dev_data = readTrainData("SemEval-PIT2015-py3/data/dev.data")
test_data = [p for p in readTestData("SemEval-PIT2015-py3/data/test.data") if p.label is not None]

In [6]:
train_data[:3]

[Phrase(original=[Token(text='ej', tags=('B-person', 'NNP', 'B-NP', 'O')), Token(text='manuel', tags=('I-person', 'NNP', 'B-VP', 'O')), Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='1st', tags=('O', 'CD', 'I-NP', 'O')), Token(text='qb', tags=('O', 'NNP', 'I-NP', 'O')), Token(text='to', tags=('O', 'TO', 'B-VP', 'O')), Token(text='go', tags=('O', 'VB', 'I-VP', 'B-EVENT')), Token(text='in', tags=('O', 'IN', 'B-PP', 'I-EVENT')), Token(text='this', tags=('O', 'DT', 'B-NP', 'O')), Token(text='draft', tags=('O', 'NN', 'I-NP', 'O'))], candidate=[Token(text='but', tags=('O', 'CC', 'O', 'O')), Token(text='my', tags=('O', 'PRP$', 'B-NP', 'O')), Token(text='bro', tags=('O', 'NN', 'I-NP', 'O')), Token(text='from', tags=('O', 'IN', 'B-PP', 'O')), Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='757', tags=('O', 'CD', 'I-NP', 'O')), Token(text='ej', tags=('B-person', 'NNP', 'I-NP', 'O')), Token(text='manuel', tags=('I-person', 'NNP', 'I-NP', 'O')), Token(text='is', tags=(

In [7]:
test_data[:3]

[Phrase(original=[Token(text='all', tags=('O', 'DT', 'B-NP', 'O')), Token(text='the', tags=('O', 'DT', 'I-NP', 'O')), Token(text='home', tags=('O', 'NN', 'I-NP', 'O')), Token(text='alones', tags=('O', 'VBZ', 'B-VP', 'O')), Token(text='watching', tags=('O', 'VBG', 'I-VP', 'B-EVENT')), Token(text='8', tags=('O', 'CD', 'B-NP', 'O')), Token(text='mile', tags=('O', 'NN', 'I-NP', 'O'))], candidate=[Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='last', tags=('O', 'JJ', 'I-NP', 'O')), Token(text='rap', tags=('O', 'NN', 'I-NP', 'B-EVENT')), Token(text='battle', tags=('O', 'NN', 'I-NP', 'B-EVENT')), Token(text='in', tags=('O', 'IN', 'B-PP', 'O')), Token(text='8', tags=('O', 'CD', 'B-NP', 'O')), Token(text='mile', tags=('O', 'NNP', 'I-NP', 'O')), Token(text='nevr', tags=('O', 'NN', 'I-NP', 'O')), Token(text='gets', tags=('O', 'VBZ', 'B-VP', 'O')), Token(text='old', tags=('O', 'JJ', 'B-NP', 'O')), Token(text='ahah', tags=('O', 'JJ', 'I-NP', 'O'))], label=False),
 Phrase(original=[To

Робимо обробку данних - видаляємо стоп-слова та замінюємо іменовані сутності на більш генералізовані поняття.

In [8]:
from nltk.corpus import stopwords
from tqdm import tqdm_notebook

stops = stopwords.words('english')

def clean_sent(sent):
    new_sent = []
    for token in sent:
        if token.tags[0].startswith('B-'):
            new_sent.append(token.tags[0].split('-')[1])
            continue
        if token.tags[0].startswith('I-') or token.text in stops:
            continue
        new_sent.append(token.text)
                            
    return new_sent

def clean_data(data):
    return [Phrase(clean_sent(phrase.original), clean_sent(phrase.candidate), phrase.label) \
            for phrase in tqdm_notebook(data)]

In [9]:
clean_train_data = clean_data(train_data)

In [10]:
clean_dev_data = clean_data(dev_data)

In [11]:
clean_test_data = clean_data(test_data)

In [425]:
clean_train_data[:3]

[Phrase(original=['person', '1st', 'qb', 'go', 'draft'], candidate=['bro', '757', 'person', '1st', 'qb', 'gone'], label=True),
 Phrase(original=['person', '1st', 'qb', 'go', 'draft'], candidate=['believe', 'person', 'went', '1st', 'qb', 'draft'], label=True),
 Phrase(original=['person', '1st', 'qb', 'go', 'draft'], candidate=['person', '1st', 'qb'], label=True)]

In [426]:
clean_dev_data[:3]

[Phrase(original=['walk', 'remember', 'definition', 'true', 'love'], candidate=['walk', 'remember', 'im', 'town', 'im', 'upset'], label=False),
 Phrase(original=['walk', 'remember', 'definition', 'true', 'love'], candidate=['walk', 'remember', 'cutest', 'thing'], label=True),
 Phrase(original=['walk', 'remember', 'definition', 'true', 'love'], candidate=['walk', 'remember', 'other', 'family', 'youre', 'welcome'], label=False)]

## Model

Ідея заключається в тому щоб попрацювати з нейромережами. Спочатку підготуємо для цього дані і зробимо векторизацію тексту.

In [12]:
from gensim.corpora import Dictionary

def phrase_tokens(phrase):
    return [token for token in (phrase.original + phrase.candidate)]
    

vocab = Dictionary([phrase_tokens(p) for p in clean_train_data + clean_dev_data + clean_test_data])

len(vocab)

9060

In [13]:
VOCAB_SIZE = len(vocab) + 1 # +1 for padding

print(VOCAB_SIZE)

9061


In [14]:
id2token = dict([(i, token)for token, i in vocab.token2id.items()])

In [15]:
def text_to_sequence(words):
    return [i + 1 for i in vocab.doc2idx(words)]

def sequence_to_text(seq):
    return [id2token[i - 1] for i in seq if i]

In [16]:
def data_to_sequences(data):
    
    encoder_seqs = []
    decoder_seqs = []
    labels = []
    
    for phrase in data:
        encoder_seqs.append(text_to_sequence([t for t in phrase.original]))
        decoder_seqs.append(text_to_sequence([t for t in phrase.candidate]))
        labels.append(phrase.label)
        
    return encoder_seqs, decoder_seqs, labels 

train_encoder_seqs, train_decoder_seqs, train_labels = data_to_sequences(clean_train_data)

print(len(train_encoder_seqs))
print(len(train_decoder_seqs))
print(len(train_labels))

11530
11530
11530


In [17]:
print(sequence_to_text(train_encoder_seqs[0]))
print(sequence_to_text(train_decoder_seqs[0]))
print(train_labels[0])

['person', '1st', 'qb', 'go', 'draft']
['bro', '757', 'person', '1st', 'qb', 'gone']
True


In [18]:
dev_encoder_seqs, dev_decoder_seqs, dev_labels = data_to_sequences(clean_dev_data)

In [19]:
test_encoder_seqs, test_decoder_seqs, test_labels = data_to_sequences(clean_test_data)

In [20]:
MAX_SEQ_LEN = max([len(seq) for seq in (train_encoder_seqs + train_decoder_seqs + \
                                       dev_encoder_seqs + dev_decoder_seqs + \
                                       test_decoder_seqs + test_encoder_seqs)])

print(MAX_SEQ_LEN)

13


In [21]:
from keras.preprocessing.sequence import pad_sequences

def padding(sequences):
    return pad_sequences(sequences, maxlen=MAX_SEQ_LEN, dtype='int32', padding='post', truncating='post')

Using TensorFlow backend.


In [22]:
import numpy as np

EMBEDDING_SIZE = 300

EMBEDDING_MATRIX = np.zeros((VOCAB_SIZE, EMBEDDING_SIZE))
  
for word, i in vocab.token2id.items():
    try:
        EMBEDDING_MATRIX[i] = numberbatch[word]
    except KeyError:
        pass

Готуємо базовий клас для подальших моделей.

In [23]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from sklearn.metrics import classification_report, accuracy_score

class SemevalModel:
    def __init__(self, left_seqs, right_seqs, labels):
        self.left_seqs = left_seqs
        self.right_seqs = right_seqs
        self.labels = labels
        
    def train(self, val_left_seqs, val_right_seqs, val_labels, batch_size, epochs):
        self.history = self.model.fit([self.left_seqs, self.right_seqs],
                                      self.labels, batch_size = batch_size, epochs = epochs, 
                                      validation_data = ([val_left_seqs, val_right_seqs], val_labels))
    
    def test(self, test_left_seqs, test_right_seqs, test_labels):
        predicted_similarity = self.model.predict([test_left_seqs, test_right_seqs])
        predicted_labels = [p.item() > 0.5 for p in predicted_similarity]
        report = classification_report(test_labels, predicted_labels, output_dict = True)
        report['accuracy'] = accuracy_score(test_labels, predicted_labels)
        return predicted_labels, predicted_similarity, report
        

### Seq2Seq

Ідея цієї моделі інспірована autoencoder архітектурою. Цікаво як вона буде працювати на нашій задачі.

In [24]:
from keras.optimizers import RMSprop

embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                            output_dim = EMBEDDING_SIZE,
                            input_length = MAX_SEQ_LEN,
                            weights = [EMBEDDING_MATRIX], trainable = False)

class Seq2SeqModel(SemevalModel):
    def __init__(self, left_seqs, right_seqs, labels):
        super().__init__(left_seqs, right_seqs, labels)
        hidden_dim = 300

        encoder_inputs = Input(shape=(MAX_SEQ_LEN, ), dtype='int32',)
        encoder_embedding = embedding_layer(encoder_inputs)
        encoder_LSTM = LSTM(hidden_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
        decoder_inputs = Input(shape=(MAX_SEQ_LEN, ), dtype='int32',)
        decoder_embedding = embedding_layer(decoder_inputs)
        decoder_LSTM = LSTM(hidden_dim, return_state=True)
        decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
        outputs = Dense(1, activation='sigmoid')(decoder_outputs)
        self.model = Model([encoder_inputs, decoder_inputs], outputs)
        self.model.compile(optimizer=RMSprop(lr=5e-5), loss='binary_crossentropy', metrics=['acc'])
        self.model.summary()    

In [32]:
seq2seq = Seq2SeqModel(padding(train_encoder_seqs), padding(train_decoder_seqs), train_labels)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 13)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 13)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 13, 300)      2718300     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 300), (None, 721200      embedding_1[4][0]                
__________

In [33]:
seq2seq.train(padding(dev_decoder_seqs), padding(dev_encoder_seqs), dev_labels, batch_size = 100, epochs = 4)

Train on 11530 samples, validate on 4142 samples
Epoch 1/4
11530/11530 [==============================] - 9s 741us/step - loss: 0.6432 - acc: 0.6534 - val_loss: 0.6448 - val_acc: 0.6451
Epoch 2/4
11530/11530 [==============================] - 7s 619us/step - loss: 0.6310 - acc: 0.6537 - val_loss: 0.6344 - val_acc: 0.6451
Epoch 3/4
11530/11530 [==============================] - 7s 617us/step - loss: 0.6088 - acc: 0.6733 - val_loss: 0.6420 - val_acc: 0.6610
Epoch 4/4
11530/11530 [==============================] - 7s 616us/step - loss: 0.5910 - acc: 0.6953 - val_loss: 0.6421 - val_acc: 0.6415


Тільки 4 епохи бо модель швидко оверфітиться. 

In [34]:
_, _, seq2seq_report = seq2seq.test(padding(dev_decoder_seqs), padding(dev_encoder_seqs), dev_labels)

seq2seq_report

{'False': {'precision': 0.6958099637083471,
  'recall': 0.7892964071856288,
  'f1-score': 0.7396107311941084,
  'support': 2672},
 'True': {'precision': 0.49324932493249324,
  'recall': 0.3727891156462585,
  'f1-score': 0.42464161177838045,
  'support': 1470},
 'micro avg': {'precision': 0.641477547078706,
  'recall': 0.641477547078706,
  'f1-score': 0.641477547078706,
  'support': 4142},
 'macro avg': {'precision': 0.5945296443204202,
  'recall': 0.5810427614159437,
  'f1-score': 0.5821261714862445,
  'support': 4142},
 'weighted avg': {'precision': 0.623920987609722,
  'recall': 0.641477547078706,
  'f1-score': 0.6278278713338669,
  'support': 4142},
 'accuracy': 0.641477547078706}

In [35]:
seq2seq_test_lablels, seq2seq_test_similarity, seq2seq_test_report = seq2seq.test(padding(test_decoder_seqs),
                                                                                  padding(test_encoder_seqs), test_labels)
seq2seq_test_report

{'False': {'precision': 0.8237885462555066,
  'recall': 0.8461538461538461,
  'f1-score': 0.8348214285714285,
  'support': 663},
 'True': {'precision': 0.3503184713375796,
  'recall': 0.3142857142857143,
  'f1-score': 0.3313253012048193,
  'support': 175},
 'micro avg': {'precision': 0.7350835322195705,
  'recall': 0.7350835322195705,
  'f1-score': 0.7350835322195703,
  'support': 838},
 'macro avg': {'precision': 0.587053508796543,
  'recall': 0.5802197802197802,
  'f1-score': 0.5830733648881239,
  'support': 838},
 'weighted avg': {'precision': 0.7249135306103548,
  'recall': 0.7350835322195705,
  'f1-score': 0.7296760559113371,
  'support': 838},
 'accuracy': 0.7350835322195705}

Бачимо ще результати гірше ніж у бейзлайна, якщо дивитися тільки на True клас.

### Manhattan LSTM

Тепер спробуємо архітектуру яка вже використовувалась на подібній задачі https://www.kaggle.com/c/quora-question-pairs. Більш детальний опис можна знайти тут https://medium.com/mlreview/implementing-malstm-on-kaggles-quora-question-pairs-competition-8b31b0b16a07 або в оригіналі https://www.aaai.org/ocs/index.php/AAAI/AAAI16/paper/download/12195/12023.

In [36]:
import keras
import keras.backend as K
from keras.layers import Lambda
from keras.optimizers import Adadelta

def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

class MaLstmModel(SemevalModel):
    def __init__(self, left_seqs, right_seqs, labels):
        super().__init__(left_seqs, right_seqs, labels)
    
        left_input = Input(shape=(MAX_SEQ_LEN,), dtype='int32')
        right_input = Input(shape=(MAX_SEQ_LEN,), dtype='int32')

        left_embedding = embedding_layer(left_input)
        right_embedding = embedding_layer(right_input)

        shared_lstm = LSTM(100)

        left_output = shared_lstm(left_embedding)
        right_output = shared_lstm(right_embedding)

        ma_distance = Lambda(function = lambda x: exponent_neg_manhattan_distance(x[0], x[1]),
                             output_shape = lambda x: (x[0][0], 1))([left_output, right_output])

        self.model = Model([left_input, right_input], [ma_distance])

        self.model.compile(optimizer = Adadelta(clipnorm = 1.25),
                           loss='mean_squared_error', metrics=['accuracy'])
        self.model.summary()

In [48]:
malstm = MaLstmModel(padding(train_encoder_seqs), padding(train_decoder_seqs), train_labels)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 13)           0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 13)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 13, 300)      2718300     input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
lstm_9 (LSTM)                   (None, 100)          160400      embedding_1[10][0]               
          

In [49]:
malstm.train(padding(dev_decoder_seqs), padding(dev_encoder_seqs), dev_labels, batch_size = 64, epochs = 5)

Train on 11530 samples, validate on 4142 samples
Epoch 1/5
11530/11530 [==============================] - 7s 608us/step - loss: 0.2448 - acc: 0.6144 - val_loss: 0.2637 - val_acc: 0.6241
Epoch 2/5
11530/11530 [==============================] - 5s 448us/step - loss: 0.2054 - acc: 0.6859 - val_loss: 0.2364 - val_acc: 0.6161
Epoch 3/5
11530/11530 [==============================] - 5s 444us/step - loss: 0.1888 - acc: 0.7140 - val_loss: 0.2312 - val_acc: 0.6265
Epoch 4/5
11530/11530 [==============================] - 5s 443us/step - loss: 0.1787 - acc: 0.7428 - val_loss: 0.2363 - val_acc: 0.5951
Epoch 5/5
11530/11530 [==============================] - 5s 445us/step - loss: 0.1706 - acc: 0.7573 - val_loss: 0.2342 - val_acc: 0.6033


In [50]:
_, _, malstm_report = malstm.test(padding(dev_decoder_seqs), padding(dev_encoder_seqs), dev_labels)

malstm_report

{'False': {'precision': 0.6793307772743116,
  'recall': 0.7294161676646707,
  'f1-score': 0.7034831257895687,
  'support': 2672},
 'True': {'precision': 0.43205027494108406,
  'recall': 0.3741496598639456,
  'f1-score': 0.4010207801676996,
  'support': 1470},
 'micro avg': {'precision': 0.6033317238049252,
  'recall': 0.6033317238049252,
  'f1-score': 0.6033317238049252,
  'support': 4142},
 'macro avg': {'precision': 0.5556905261076979,
  'recall': 0.5517829137643081,
  'f1-score': 0.5522519529786342,
  'support': 4142},
 'weighted avg': {'precision': 0.5915706762531033,
  'recall': 0.6033317238049252,
  'f1-score': 0.5961389326306726,
  'support': 4142},
 'accuracy': 0.6033317238049252}

In [57]:
malstm_test_lablels, malstm_test_similarity, malstm_test_report = malstm.test(padding(test_decoder_seqs),
                                                                              padding(test_encoder_seqs), test_labels)
malstm_test_report

{'False': {'precision': 0.8171334431630972,
  'recall': 0.7481146304675717,
  'f1-score': 0.7811023622047244,
  'support': 663},
 'True': {'precision': 0.27705627705627706,
  'recall': 0.3657142857142857,
  'f1-score': 0.31527093596059114,
  'support': 175},
 'micro avg': {'precision': 0.6682577565632458,
  'recall': 0.6682577565632458,
  'f1-score': 0.6682577565632458,
  'support': 838},
 'macro avg': {'precision': 0.5470948601096871,
  'recall': 0.5569144580909287,
  'f1-score': 0.5481866490826578,
  'support': 838},
 'weighted avg': {'precision': 0.7043488321026037,
  'recall': 0.6682577565632458,
  'f1-score': 0.6838225297551739,
  'support': 838},
 'accuracy': 0.6682577565632458}

In [ ]:
Бачимо що результати приблизно однакові, але все одно гірше бейзлайна.

### Universal Sentence Encoder

Спробуємо використати transfer learning. Для цього візьмемо https://tfhub.dev/google/universal-sentence-encoder/2 яка обіцяє покращення на аналогічних задачах.

In [25]:
import numpy as np

def originals(data):
    return np.array([' '.join(p.original) for p in data])
                    
def candidates(data):
    return np.array([' '.join(p.candidate) for p in data])

In [26]:
import tensorflow as tf
import tensorflow_hub as hub

embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")

W0528 14:10:36.570930 139747452421952 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Instructions for updating:
Colocations handled automatically by placer.


W0528 14:10:37.480704 139747452421952 deprecation.py:323] From /home/szubovych/.virtualenvs/nlp/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [27]:
def universal_embedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
                 signature="default", as_dict=True)["default"]

In [45]:
from keras.layers import concatenate

class UniversalModel(SemevalModel):
    def __init__(self, left_seqs, right_seqs, labels):
        super().__init__(left_seqs, right_seqs, labels)
    
        left_input = Input(shape=(1,), dtype=tf.string)
        right_input = Input(shape=(1,), dtype=tf.string)

        left_embedding = Lambda(universal_embedding, output_shape=(512,))(left_input)
        right_embedding = Lambda(universal_embedding, output_shape=(512,))(right_input)

        left_hidden = Dense(512, activation = "relu")(left_embedding)
        right_hidden = Dense(512, activation = "relu")(right_embedding)
        
        concats = concatenate([left_hidden, right_hidden], axis=-1)
        
        hidden = Dense(256, activation = "relu")(concats)
        
        out = Dense(1, activation = "sigmoid")(hidden)

        self.model = Model([left_input, right_input], [out])
        
        self.model.compile(optimizer = RMSprop(lr=5e-5),
                           loss='binary_crossentropy', metrics=['accuracy'])
        self.model.summary()

In [55]:
universal = UniversalModel(originals(clean_train_data), candidates(clean_train_data), train_labels)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 17:31:35.703430 139747452421952 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 17:31:36.802050 139747452421952 saver.py:1483] Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, 512)          0           input_11[0][0]                   
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, 512)          0           input_12[0][0]                   
__________________________________________________________________________________________________
dense_21 (

In [56]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    universal.train(originals(clean_dev_data), candidates(clean_dev_data), dev_labels,
                      batch_size = 64, epochs=6)
    
    universal_result = universal.test(originals(clean_dev_data), candidates(clean_dev_data), dev_labels)
    
    test_universal_result = universal.test(originals(clean_test_data), candidates(clean_test_data), test_labels)

Train on 11530 samples, validate on 4142 samples
Epoch 1/6
11530/11530 [==============================] - 75s 7ms/step - loss: 0.6307 - acc: 0.6546 - val_loss: 0.6388 - val_acc: 0.6456
Epoch 2/6
11530/11530 [==============================] - 66s 6ms/step - loss: 0.5680 - acc: 0.7144 - val_loss: 0.6299 - val_acc: 0.6552
Epoch 3/6
11530/11530 [==============================] - 68s 6ms/step - loss: 0.5275 - acc: 0.7599 - val_loss: 0.6267 - val_acc: 0.6579
Epoch 4/6
11530/11530 [==============================] - 67s 6ms/step - loss: 0.5037 - acc: 0.7722 - val_loss: 0.6325 - val_acc: 0.6637
Epoch 5/6
11530/11530 [==============================] - 66s 6ms/step - loss: 0.4875 - acc: 0.7825 - val_loss: 0.6209 - val_acc: 0.6779
Epoch 6/6
11530/11530 [==============================] - 64s 6ms/step - loss: 0.4741 - acc: 0.7892 - val_loss: 0.6230 - val_acc: 0.6782


In [59]:
universal_result[2]

{'False': {'precision': 0.6884323107233324,
  'recall': 0.9154191616766467,
  'f1-score': 0.785863453815261,
  'support': 2672},
 'True': {'precision': 0.6162988115449916,
  'recall': 0.24693877551020407,
  'f1-score': 0.3525983487129674,
  'support': 1470},
 'micro avg': {'precision': 0.678174794785128,
  'recall': 0.678174794785128,
  'f1-score': 0.678174794785128,
  'support': 4142},
 'macro avg': {'precision': 0.652365561134162,
  'recall': 0.5811789685934254,
  'f1-score': 0.5692309012641142,
  'support': 4142},
 'weighted avg': {'precision': 0.6628320587213622,
  'recall': 0.678174794785128,
  'f1-score': 0.6320972286823852,
  'support': 4142},
 'accuracy': 0.678174794785128}

In [60]:
test_universal_result[2]

{'False': {'precision': 0.8144192256341789,
  'recall': 0.9200603318250377,
  'f1-score': 0.8640226628895185,
  'support': 663},
 'True': {'precision': 0.4044943820224719,
  'recall': 0.2057142857142857,
  'f1-score': 0.2727272727272727,
  'support': 175},
 'micro avg': {'precision': 0.7708830548926014,
  'recall': 0.7708830548926014,
  'f1-score': 0.7708830548926014,
  'support': 838},
 'macro avg': {'precision': 0.6094568038283255,
  'recall': 0.5628873087696616,
  'f1-score': 0.5683749678083956,
  'support': 838},
 'weighted avg': {'precision': 0.7288143955243355,
  'recall': 0.7708830548926014,
  'f1-score': 0.7405421219845149,
  'support': 838},
 'accuracy': 0.7708830548926014}

In [ ]:
Бачимо що покращення не відбулось, рузультати гірші ніж у двох попередніх моделей. Думаю модель просто нічого не знає про Twitter.

## Summary

Проженемо тестовий скрипт і підсумуємо результати.

In [53]:
def dump_scores(filename, predicted_similarity):
    with open(filename, 'w+') as f:
        for estimate in predicted_similarity:                    
            f.write("{}\t{:.4f}\n".format(str(estimate.item() > 0.5).lower(), estimate.item()))

Оскільки ми не брали до уваги debatable клас то його треба відфільтрувати з еталонного файлу інакше скрипт не спрацює.

In [109]:
!cat SemEval-PIT2015-py3/data/test.label | grep -E "true|false" > test_bin.label

### Baseline

Виведемо для порівняння бейзлайн.

In [65]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py SemEval-PIT2015-py3/data/test.label SemEval-PIT2015-py3/systemoutputs/PIT2015_BASELINE_02_LG.output

838	BASELINE	02_LG		F: 0.589	Prec: 0.679	Rec: 0.520		P-corr: 0.511	F1: 0.601	Prec: 0.674	Rec: 0.543


### Seq2Seq

In [54]:
dump_scores('PIT2015_zubovych_autoencoder.output', seq2seq_test_similarity)

In [55]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py test_bin.label PIT2015_zubovych_autoencoder.output

838	zubovych	autoencoder		F: 0.331	Prec: 0.350	Rec: 0.314		P-corr: 0.227	F1: 0.383	Prec: 0.278	Rec: 0.617


### Manhattan LSTM

In [58]:
dump_scores('PIT2015_zubovych_malstm.output', malstm_test_similarity)

In [59]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py test_bin.label PIT2015_zubovych_malstm.output

838	zubovych	malstm		F: 0.315	Prec: 0.277	Rec: 0.366		P-corr: 0.170	F1: 0.375	Prec: 0.240	Rec: 0.857


### Universal Sentence Encoder

In [70]:
dump_scores('PIT2015_zubovych_universal.output',  test_universal_result[1])

In [72]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py test_bin.label PIT2015_zubovych_universal.output

838	zubovych	universal		F: 0.273	Prec: 0.404	Rec: 0.206		P-corr: 0.255	F1: 0.394	Prec: 0.356	Rec: 0.440


In [ ]:
Бейзлайн не побили. Autoencoder та MaLSTM, яка на quora датасеті дає гарну якість, виглядають приблизно однаково.
Слід зауважити що датасет від quora в разів 30 більший і це не твітер, де в тексті дуже багато опечаток і скороченнь.
Хочу додати що ConceptNet Numberbatch дав кращі результати порівняно зі звичайним word2vec, glove і fasttext 
(не всі експерименти були включені в фінальний notebook)